In [1]:
# pip install neo4j
from neo4j import GraphDatabase

In [2]:
# нашёл этот способ на Хабре
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self.driver is not None:
            self.driver.close()

# Метод, который передает запрос в БД
    def query(self, query, db=None):
        assert self.driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.driver.session(database=db) if db is not None else self.driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [3]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user='test', password='password')
conn.query('CREATE OR REPLACE DATABASE neo4j')

[]

In [6]:
query_string = '''
LOAD CSV WITH HEADERS FROM 
'file:///newone.csv'
AS line FIELDTERMINATOR ';'
MERGE (ev:event {eventID: toInteger(line.EventID)})
MERGE (person1:personOne {name: line.NameOne})
MERGE (person2:personTwo {name: line.NameTwo})
ON CREATE SET 
ev.personOne = line.NameOne, ev.personTwo = line.NameTwo, 
person1.eventID = toInteger(line.EventID),  
person2.eventID = toInteger(line.EventID);
'''

conn.query(query_string, db='neo4j')

[]

In [7]:
query_string = '''
MATCH (e:event), (p1:personOne), (p2:personTwo)
WHERE e.eventID = p1.eventID = p2.eventID
CREATE (p1)-[:TOOK_PART]->(e)<-[:TOOK_PART]-(p2);
'''
conn.query(query_string, db='neo4j')

[]

In [156]:
# на случай обнуления всей базы
query_string='''
MATCH (n)
DETACH DELETE n
'''
conn.query(query_string, db='test')

[]

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('../User/newone.csv', sep=';')

In [10]:
df.head()

,EventID,NameOne,NameTwo
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна


In [8]:
print('Всего событий:', len(df['EventID']))
print('Всего уникальный событий:', len(df['EventID'].unique()))

Всего событий: 5000
Всего уникальный событий: 4985


In [9]:
print('Уникальных первых участников', len(df['NameOne'].unique()))
print('Уникальных вторых участников', len(df['NameTwo'].unique()))

Уникальных первых участников 4930
Уникальных вторых участников 4994


In [114]:
# Словарь содержит количество по каждому участнику 1
took_parts_one = {}
po = df['NameOne']
for i in range(len(po)):
    took_parts_one[po[i]] = took_parts_one.get(po[i], 0) + 1
for key, value in took_parts_one.items():
    if value > 1:
        print('Участник №1:', key,'| Событий:', value)

Участник №1: Ахромеева Алина Ивановна | Событий: 49
Участник №1: Башнина Антонина Глебовна | Событий: 14
Участник №1: Пафомова Кира Вадимовна | Событий: 2
Участник №1: Зимнухова Карина Даниловна | Событий: 4
Участник №1: Шолохов Игорь Робертович | Событий: 2
Участник №1: Диомидов Игорь Ильдарович | Событий: 5


In [22]:
df.keys()

Index(['EventID', 'NameOne', 'NameTwo'], dtype='object')

In [23]:
ahro_events = []
index = 0
for ev_id in (df['EventID']):
    if df['NameOne'][index] == 'Ахромеева Алина Ивановна':
        ahro_events.append((ev_id))
    index += 1

In [41]:
# Словарь содержит количество по каждому участнику 2
took_parts_two = {}
pt = df['NameTwo']
for i in range(len(pt)):
    took_parts_two[pt[i]] = took_parts_two.get(pt[i], 0) + 1
for key, value in took_parts_two.items():
    if value > 1:
        print('Участник №2:', key,'| Событий:', value)

Участник №2: Медведева Дарья Алексеевна | Событий: 5
Участник №2: Двигубская Валентина Геннадьевна | Событий: 2
Участник №2: Шолохов Игорь Робертович | Событий: 2


In [110]:
# Словарь содержит количество по каждому событию
event_count = {}
ev = df['EventID']
for i in range(len(ev)):
    event_count[ev[i]] = event_count.get(ev[i], 0) + 1
for key, value in event_count.items():
    if value > 1:
        print('Событие eventID', key,'| Всего', value)

Событие eventID 70049 | Всего 2
Событие eventID 92995 | Всего 2
Событие eventID 117280 | Всего 2
Событие eventID 177407 | Всего 2
Событие eventID 358194 | Всего 2
Событие eventID 390312 | Всего 2
Событие eventID 523688 | Всего 2
Событие eventID 551592 | Всего 2
Событие eventID 613539 | Всего 2
Событие eventID 716489 | Всего 2
Событие eventID 765223 | Всего 2
Событие eventID 850472 | Всего 2
Событие eventID 873359 | Всего 2
Событие eventID 938764 | Всего 2
Событие eventID 985851 | Всего 2


In [52]:
# Всего повторяющихся событий
total_events_twice_count = 0
total_events_twice_dict = {}
for key, value in event_count.items():
    if value > 1:
        total_events_twice_count += 1
        if key in total_events_twice_dict:
            total_events_twice_dict[key].append()
        elif key not in total_events_twice_dict:
            total_events_twice_dict[key] = total_events_twice_dict.get(key, [value])
print(f'Всего повторяющихся событий {total_events_twice_count}')
print(f'Это {total_events_twice/5000*100}% от общего числа событий')

Всего повторяющихся событий 15
Это 0.3% от общего числа событий


In [87]:
# Выведем участников в повторяющихся событиях
for key, value in df['EventID'].items():
    if value in total_events_twice_dict:
        print(f"id {str(value).ljust(6)}: {df['NameOne'][key].ljust(31)} | {df['NameTwo'][key]}")

id 70049 : Яшина Полина Евгеньевна         | Герасимовская Ксения Дамировна
id 70049 : Федова Анжелика Вадимовна       | Вальдовский Альберт Ефимович
id 92995 : Журик Альберт Евгеньевич        | Бадьянова Римма Максимовна
id 92995 : Кучеренко Ирина Ильинична       | Болтик Григорий Максимович
id 117280: Волынский Кирилл Федорович      | Гайсумов Виктор Тимурович
id 117280: Уточкин Евгений Анатольевич     | Каганович Лилия Петровна
id 177407: Зелинский Геннадий Артурович    | Зазорин Вадим Аркадьевич
id 177407: Бужанинов Руслан Артурович      | Сороковой Герман Маратович
id 358194: Ноева Галина Степановна         | Серпухова Алла Ярославовна
id 358194: Долгих Лилия Вадимовна          | Брусенцова Дарья Михаиловна
id 390312: Григорьевых Павел Леонидович    | Хилин Федор Федорович
id 390312: Намазова Евгения Дмитриевна     | Охоцимская Виктория Евгеньевна
id 523688: Бацких Егор Олегович            | Барилов Роман Филиппович
id 523688: Ботяновская Антонина Даниловна  | Памфилова Тамара Дан

In [102]:
# Словарь повторящихся событий и списком кортежей их участников
double_events_participants = {}
for key, value in df['EventID'].items():
    if value in total_events_twice_dict:
        if value not in double_events_participants.keys():
            double_events_participants[value] = double_events_participants.get(value, [(df['NameOne'][key], df['NameTwo'][key])])
#         print(f"id {str(value).ljust(6)}: {df['NameOne'][key].ljust(31)} | {df['NameTwo'][key]}")
        elif value in double_events_participants.keys():
            double_events_participants[value].append((df['NameOne'][key], df['NameTwo'][key]))

In [11]:
from flask import Flask, jsonify
api = Flask(__name__)
@api.route("/create/<string:name>", methods=["GET", "POST"])
def create_node(name):
    query_string = """
    CREATE (n:Emloyee {NAME:$name})
    """
    map = {"name": name}
    try:
        conn.query(query_string, db='test')
        return "OK"
    except Exception as e:
        return (str(e))